
# Prediksi Risiko Stroke dengan Jaringan Saraf Tiruan + Antarmuka Streamlit

Notebook ini membangun model **Jaringan Saraf Tiruan (Artificial Neural Network / ANN)**  
untuk memprediksi risiko stroke menggunakan **dataset stroke**, lalu menyiapkan kode
**antarmuka Streamlit** seperti contoh sistem pendukung keputusan penyakit jantung
(dengan slider dan dropdown).

## Alur Notebook

1. Import library & load dataset stroke.  
2. Eksplorasi singkat & penanganan *missing value*.  
3. Pra-pemrosesan data (encoding kategorik + scaling numerik).  
4. Split data menjadi train / test.  
5. Membangun & melatih model **ANN (MLPClassifier)**.  
6. Evaluasi model (akurasi, confusion matrix, ROC-AUC).  
7. Menyimpan model terlatih ke file `.pkl`.  
8. Contoh kode **Streamlit** dengan slider & selectbox untuk prediksi stroke secara interaktif.

> Semua bagian kode diberi komentar dan penjelasan dalam bahasa Indonesia.



## 1. Import Library dan Load Dataset Stroke

Pada bagian ini:
- Kita mengimpor library yang diperlukan.  
- Membaca dataset stroke dari file `.csv`.  
- Menampilkan beberapa baris teratas untuk melihat struktur data.


In [ ]:

# Import library dasar untuk analisis dan visualisasi data
import pandas as pd          # untuk manipulasi data berbasis tabel
import numpy as np           # untuk perhitungan numerik
import matplotlib.pyplot as plt  # untuk visualisasi (jika dibutuhkan)

# Agar grafik muncul langsung di notebook
%matplotlib inline

# Path dataset stroke (file sudah di-upload)
csv_path = "/mnt/data/healthcare-dataset-stroke-data.csv"

# Membaca dataset ke DataFrame pandas
df = pd.read_csv(csv_path)

# Menampilkan 5 baris pertama untuk melihat struktur kolom
df.head()



## 2. Eksplorasi Data & Cek Missing Value

Kita perlu memahami:
- Tipe data tiap kolom.  
- Distribusi dasar dari fitur numerik.  
- Apakah ada nilai kosong (*missing values*).  


In [ ]:

# Informasi struktur data: jumlah baris/kolom dan tipe data
df.info()


In [ ]:

# Ringkasan statistik dasar untuk kolom numerik
df.describe()


In [ ]:

# Mengecek jumlah nilai kosong pada setiap kolom
df.isnull().sum()



### 2.1 Menangani Missing Value pada `bmi`

Biasanya kolom `bmi` memiliki beberapa nilai kosong (`NaN`).  
Strategi sederhana yang digunakan:
- Mengisi `NaN` dengan **median** kolom `bmi`.  
  - Median cukup *robust* terhadap outlier.  


In [ ]:

# Menghitung median untuk kolom 'bmi'
bmi_median = df['bmi'].median()

# Mengisi nilai NaN pada 'bmi' dengan median
df['bmi'] = df['bmi'].fillna(bmi_median)

# Memastikan tidak ada lagi nilai kosong di kolom 'bmi'
df['bmi'].isnull().sum()



## 3. Menentukan Fitur (X) dan Target (y)

- **Target (y)**: kolom `stroke`  
  - `1` → pasien mengalami stroke / berisiko stroke.  
  - `0` → tidak stroke.  

- **Fitur (X)**: semua kolom selain `id` dan `stroke`.  
  - Kolom `id` hanya sebagai identitas unik → dibuang.


In [ ]:

# Menghapus kolom 'id' karena tidak informatif untuk prediksi
df_model = df.drop(columns=['id'])

# Memisahkan fitur (X) dan target (y)
X = df_model.drop(columns=['stroke'])
y = df_model['stroke']

# Menampilkan nama-nama kolom fitur
X.columns



### 3.1 Memisahkan Fitur Numerik dan Kategorik

Supaya pra-pemrosesan lebih rapi:
- Fitur **numerik** akan di-*scale* dengan `StandardScaler`.  
- Fitur **kategorik** akan di-*encode* dengan `OneHotEncoder`.  

Contoh di dataset stroke:
- Numerik: `age`, `avg_glucose_level`, `bmi`.  
- Kategorik: `gender`, `ever_married`, `work_type`, `Residence_type`, `smoking_status`.  
  (Kolom 0/1 seperti `hypertension` dan `heart_disease` tetap dianggap numerik sederhana).


In [ ]:

# Menentukan fitur numerik dan kategorik
numeric_features = ['age', 'avg_glucose_level', 'bmi']

# Fitur lain (selain target dan numerik) kita perlakukan sebagai kategorik
all_features = list(X.columns)
categorical_features = [col for col in all_features if col not in numeric_features]

numeric_features, categorical_features



## 4. Pra-pemrosesan & Model ANN dalam Pipeline

Kita menggunakan `ColumnTransformer` dan `Pipeline` dari scikit-learn:

- `ColumnTransformer`:
  - Menerapkan `StandardScaler` ke fitur numerik.
  - Menerapkan `OneHotEncoder` ke fitur kategorik.

- `Pipeline`:
  - Langkah 1: pra-pemrosesan (`ColumnTransformer`).  
  - Langkah 2: model **ANN** (`MLPClassifier`).  

Dengan cara ini, semua proses (encoding, scaling, training) terbungkus rapi dalam satu objek.


In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve

# Transformer untuk fitur numerik: standarisasi
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Transformer untuk fitur kategorik: One-Hot Encoding
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Menggabungkan keduanya dalam ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)



## 5. Membagi Data Menjadi Train dan Test

Untuk mengukur performa model pada data yang belum dilihat, kita membagi data:

- 80% → data latih (`train`).  
- 20% → data uji (`test`).  

Kita gunakan `stratify=y` agar proporsi kelas (0 dan 1) tetap seimbang di train dan test.


In [ ]:

# Split data menjadi train dan test
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

len(X_train), len(X_test)



## 6. Membangun Model Jaringan Saraf Tiruan (MLPClassifier)

Kita gunakan `MLPClassifier` dengan konfigurasi:

- `hidden_layer_sizes=(32, 16)` → 2 hidden layer (32 neuron & 16 neuron).  
- `activation='relu'` → fungsi aktivasi ReLU.  
- `solver='adam'` → optimizer Adam.  
- `max_iter=300` → maksimal 300 iterasi training.  

Model ini digabung dengan `preprocessor` dalam `Pipeline` bernama `clf`.


In [ ]:

# Definisi model ANN (MLPClassifier)
mlp_clf = MLPClassifier(
    hidden_layer_sizes=(32, 16),
    activation='relu',
    solver='adam',
    max_iter=300,
    random_state=42
)

# Pipeline: pra-pemrosesan + model ANN
clf = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('model', mlp_clf)
])

clf



### 6.1 Melatih Model ANN

Sekarang kita latih model pada data train (`X_train`, `y_train`).  
Pra-pemrosesan (scaling + encoding) akan otomatis dijalankan di dalam `Pipeline`.


In [ ]:

# Melatih model
clf.fit(X_train, y_train)



## 7. Evaluasi Model

Kita hitung:
- Akurasi pada data train dan test.  
- Confusion Matrix dan Classification Report pada data test.  
- Nilai ROC-AUC dan kurva ROC.  


In [ ]:

# Prediksi pada data train dan test
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

# Akurasi
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print("Akurasi data latih :", train_acc)
print("Akurasi data uji   :", test_acc)


In [ ]:

# Confusion matrix & classification report untuk data uji
cm = confusion_matrix(y_test, y_test_pred)
report = classification_report(y_test, y_test_pred, digits=3)

print("Confusion Matrix (data uji):")
print(cm)
print("\nClassification Report (data uji):")
print(report)


In [ ]:

# ROC-AUC dan kurva ROC
y_test_proba = clf.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_test_proba)
print("ROC-AUC (data uji):", roc_auc)

fpr, tpr, thresholds = roc_curve(y_test, y_test_proba)

plt.figure()
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Kurva ROC - Prediksi Stroke (ANN)')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()



## 8. Menyimpan Model ke File `.pkl`

Agar bisa dipakai oleh aplikasi **Streamlit**, model `clf` (Pipeline lengkap)
akan disimpan ke file dengan format pickle (`.pkl`) menggunakan `joblib`.


In [ ]:

import joblib

# Menyimpan pipeline + model ke file
model_path = "stroke_ann_pipeline.pkl"
joblib.dump(clf, model_path)

model_path



## 9. Kode Aplikasi Streamlit (UI Slider & Dropdown)

Bagian ini berisi **contoh kode Streamlit** sehingga tampilannya mirip dengan
_Sistem Penunjang Keputusan_ untuk penyakit jantung yang menggunakan slider dan dropdown.

### Cara Pakai

1. Pastikan file model `stroke_ann_pipeline.pkl` sudah dibuat oleh notebook ini.  
2. Buat file Python baru, misalnya **`app_stroke.py`**.  
3. Salin kode di bawah ke dalam `app_stroke.py`.  
4. Di terminal, jalankan:

```bash
streamlit run app_stroke.py
```

5. Browser akan terbuka menampilkan form input (umur, jenis kelamin, hipertensi, dll) dan tombol prediksi.


In [ ]:

# ======== KODE UNTUK FILE: app_stroke.py ========
# Jika dijalankan di notebook, kode ini hanya sebagai contoh teks.
# Untuk benar-benar menjalankan Streamlit:
# 1. Salin kode ini ke file baru bernama `app_stroke.py`
# 2. Jalankan: streamlit run app_stroke.py

import streamlit as st
import pandas as pd
import joblib

# Memuat model pipeline yang sudah dilatih
model = joblib.load("stroke_ann_pipeline.pkl")

st.title("🧠 Sistem Penunjang Keputusan Prediksi Stroke")

st.markdown("Masukkan data pasien pada form berikut, lalu klik **Prediksi**.")

# --- Input fitur dengan slider dan dropdown ---

# Umur
age = st.slider("Umur", min_value=0, max_value=100, value=45)

# Jenis kelamin (sesuaikan dengan kategori di dataset: 'Male', 'Female', 'Other')
gender = st.selectbox("Jenis Kelamin", ["Male", "Female", "Other"])

# Hipertensi (0 = tidak, 1 = ya)
hypertension_label = st.selectbox("Hipertensi", ["Tidak", "Ya"])
hypertension = 1 if hypertension_label == "Ya" else 0

# Penyakit jantung (0 = tidak, 1 = ya)
heart_label = st.selectbox("Penyakit Jantung", ["Tidak", "Ya"])
heart_disease = 1 if heart_label == "Ya" else 0

# Pernah menikah
ever_married = st.selectbox("Status Pernikahan", ["Yes", "No"])

# Tipe pekerjaan (sesuai dataset)
work_type = st.selectbox(
    "Jenis Pekerjaan",
    ["Private", "Self-employed", "Govt_job", "children", "Never_worked"]
)

# Tipe tempat tinggal
Residence_type = st.selectbox(
    "Tipe Tempat Tinggal",
    ["Urban", "Rural"]
)

# Rata-rata kadar glukosa
avg_glucose_level = st.slider(
    "Rata-rata Kadar Glukosa",
    min_value=50.0, max_value=300.0, value=120.0
)

# BMI
bmi = st.slider(
    "BMI (Body Mass Index)",
    min_value=10.0, max_value=60.0, value=25.0
)

# Status merokok (sesuai dataset)
smoking_status = st.selectbox(
    "Status Merokok",
    ["formerly smoked", "never smoked", "smokes", "Unknown"]
)

# Tombol prediksi
if st.button("Prediksi Risiko Stroke"):
    # Menyusun data input ke dalam DataFrame satu baris
    input_dict = {
        "gender": [gender],
        "age": [age],
        "hypertension": [hypertension],
        "heart_disease": [heart_disease],
        "ever_married": [ever_married],
        "work_type": [work_type],
        "Residence_type": [Residence_type],
        "avg_glucose_level": [avg_glucose_level],
        "bmi": [bmi],
        "smoking_status": [smoking_status],
    }

    input_df = pd.DataFrame(input_dict)

    # Menggunakan model pipeline untuk prediksi
    pred = model.predict(input_df)[0]
    proba = model.predict_proba(input_df)[0, 1]

    st.subheader("Hasil Prediksi")

    if pred == 1:
        st.error(f"⚠️ Pasien **BERISIKO STROKE**.\n\nProbabilitas stroke: **{proba:.2%}**")
    else:
        st.success(f"✅ Pasien **TIDAK BERISIKO STROKE** (menurut model).\n\nProbabilitas stroke: **{proba:.2%}**")

    st.caption("Catatan: Ini hanya model pembelajaran mesin sebagai contoh akademik, "
               "bukan pengganti diagnosa dokter.")



## 10. Ringkasan

Di notebook ini, kita telah:

1. Mengolah dataset stroke (mengisi `bmi` yang kosong).  
2. Menyiapkan fitur numerik dan kategorik, lalu melakukan scaling & encoding.  
3. Membangun dan melatih model **Jaringan Saraf Tiruan (ANN)** dengan `MLPClassifier`.  
4. Mengevaluasi performa model dengan berbagai metrik (akurasi, ROC-AUC).  
5. Menyimpan pipeline + model ke file `stroke_ann_pipeline.pkl`.  
6. Menyusun kode aplikasi **Streamlit** dengan slider dan dropdown sehingga bisa menjadi
   **Sistem Penunjang Keputusan Prediksi Stroke** seperti contoh penyakit jantung yang kamu kirim.

Kamu bisa menyesuaikan teks, tampilan, bahkan menambah grafik/gauge di Streamlit sesuai kebutuhan skripsi atau proyekmu. 💡
